<table style="width:100%">
  <tr style="background-color:#FFFFFF">
    <td style="width:10%">
        <img src="https://play-lh.googleusercontent.com/K9FC23HoIYy_idt3LL26zrzMchOGORVzLNmi3L5u-MaghdjpdgHAWv7wQlscNmo9wYjC=s180-rw" alt="DiDi" width=120></td>
    <td style="width:10%">
        <img src="https://logodownload.org/wp-content/uploads/2019/05/99-app-logo.png" alt="99" width=120></td>
      <td>
          <h1 style="text-align:left; text-indent :2em;">DATA TRANSFORMATION • PART 2 (addendum)</h1>
          <h2 style="text-align:left; text-indent :2.4em;">ISIS SANTOS COSTA • 2020</h2>
      </td>
  </tr>
</table>

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Resuming: retrieving the DFs

In [1]:
import pandas as pd

In [2]:
df_orders = pd.read_csv('df_orders_t2.csv') # reading transformed data
df_trips = pd.read_csv('df_trips_t2.csv') # reading transformed data

In [3]:
df_orders.drop(df_orders.columns[0], axis = 1, inplace = True)
df_orders.head(2)

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type,pickup_date,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD,2014-04-11,Friday,15,0,18,1
1,6433698,5234567812367422,2014-04-12 00:20:28,-73.958403,40.713247,CRD,2014-04-12,Saturday,15,1,0,0


In [4]:
df_trips.drop(df_trips.columns[0], axis = 1, inplace = True)
df_trips.head(2)

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_type,dropoff_longitude,dropoff_latitude,payment_type,...,passenger_id,pickup_date,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,duration,duration_minutes,duration_type
0,718300,2014-05-24 09:08:43,2014-05-24 09:19:06,2.1,-73.967172,40.769851,Standard,-73.995022,40.765441,CRD,...,5234567815585627,2014-05-24 00:00:00,Saturday,21,1,9,1,0 days 00:10:23,10.383333,10-20min
1,14054197,2014-04-10 15:14:12,2014-04-10 15:49:05,12.4,-73.991044,40.749704,Standard,-73.909173,40.823500,CSH,...,5234567812372271,2014-04-10 00:00:00,Thursday,15,0,15,1,0 days 00:34:53,34.883333,30-40min


## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Feature engineering: datetime 📅 (cont'd)

### 📈 Adding further new features for better analysis

<i> Knowing our date span, it is now possible to think of what transformations/binnings could render meaningful insights:
    <ul>
        <li> Date [without time, may be useful for some visualizations]: gives information on the trend within the period </li>
        <li> <b> [New!] Month</b>: gives information on the trend within the period </li>
        <li> Day of the week: to get information on the seasonality along the week  </li>
        <li> Week: as the time span of data is short, weeks seem a useful aggregation </li>
        <li> <b> [New!] Week in the month</b>: gives information on the trend within the period </li>
        <li> Working day / Weekend: to get information on these chunks for seasonality </li>
        <li style="color:gray"> Period of the day: it would be interesting to have information by period (like 'early morning', 'noon', etc.), but... as NY is always busy, rush/no rush may be more informative. Let's have also the time by just the hour. </li>
        <li> Hour of the day </li>
        <li> Rush / no rush: reported as morning: 7:00 a.m. to 10:00 a.m., afternoon: 3:00 p.m. to 8:00 p.m.  </li>
        <li> Duration of the trip </li>

### Orders

#### Month

In [15]:
df_orders['pickup_month'] = pd.DatetimeIndex(df_orders['pickup_datetime']).month
100 * df_orders['pickup_month'].value_counts() / len(df_orders)

4    48.729200
3    40.914707
5    10.356093
Name: pickup_month, dtype: float64

🤔 <i> Hm... the number of orders in May seems too low. What about the trips? And the trend in weeks? Let's check it just below. </i>

#### Week in the month

In [31]:
df_orders['pickup_weekinmonth'] = ((pd.DatetimeIndex(df_orders['pickup_datetime']).day - 1) / 7).astype(int) + 1
100 * df_orders['pickup_weekinmonth'].value_counts() / len(df_orders)

2    25.114200
3    24.640587
1    23.455200
4    19.902400
5     6.887613
Name: pickup_weekinmonth, dtype: float64

\[ L I G H T B U L B \] <i> Interesting: there is a lower demand at the end of the month. To be checked for converted trips. </i>

\[ !!! W A R N I N G !!! \] <i> Week 5 is affected by having less days, requiring normalization. <br>
    In this case, for aggregate, the weighted average for the three months will be applied: <br>
    <ul>
        <li> Mar 40.914707 %  (w5 = 3d) </li>
        <li> Apr 48.729200 %  (w5 = 2d) </li>
        <li> May 10.356093 %  (w5 = 3d) </li>
    </ul>
</i>

In [40]:
days_w5 = 3*(0.40914707+0.10356093) + 2*(0.48729200)
print(' Average days in week 5:', days_w5, '\n\n Distribution of frequency by week of the month:')
100 * df_orders['pickup_weekinmonth'].value_counts() / len(df_orders) * [1,1,1,1,7/days_w5]

 Average days in week 5: 2.512708 

 Distribution of frequency by week of the month:


2    25.114200
3    24.640587
1    23.455200
4    19.902400
5    19.187782
Name: pickup_weekinmonth, dtype: float64

### Trips

#### Month

In [41]:
df_trips['pickup_month'] = pd.DatetimeIndex(df_trips['pickup_datetime']).month
100 * df_trips['pickup_month'].value_counts() / len(df_trips)

4    48.734797
3    40.906627
5    10.358576
Name: pickup_month, dtype: float64

<i> For trips, also a low the number of orders in May as seen for orders. Really too low. To be better checked in EDA. </i> \[ !!! W A R N I N G !!! \] 

#### Week in the month

In [42]:
df_trips['pickup_weekinmonth'] = ((pd.DatetimeIndex(df_trips['pickup_datetime']).day - 1) / 7).astype(int) + 1
100 * df_trips['pickup_weekinmonth'].value_counts() / len(df_trips)

2    25.101702
3    24.640975
1    23.453716
4    19.908946
5     6.894661
Name: pickup_weekinmonth, dtype: float64

In [44]:
days_w5 = 3*(0.40906627+0.10358576) + 2*(0.48734797)
print(' Normalized \n\n Average days in week 5:', days_w5, '\n\n Distribution of frequency by week of the month:')
100 * df_trips['pickup_weekinmonth'].value_counts() / len(df_trips) * [1,1,1,1,7/days_w5]

 Normalized 

 Average days in week 5: 2.5126520300000004 

 Distribution of frequency by week of the month:


2    25.101702
3    24.640975
1    23.453716
4    19.908946
5    19.207843
Name: pickup_weekinmonth, dtype: float64

## Exporting results for retrieval

In [46]:
df_orders.to_csv('df_orders_t3.csv')

In [47]:
df_trips.to_csv('df_trips_t3.csv')

<i>Note: This Notebook would entail the whole transforming process, but as some instabilities may interrupt the Kernel requiring a restart from zero, it has been split for a smoothier process.</i>

# Next: [Continues as Transformation, part 3](99__DataTransformation3F.ipynb)